In [1]:
from pathlib import Path
import os
import pandas as pd
import pyperclip
import subprocess
from reload_recursive import reload_recursive

import mri_data

In [31]:
reload_recursive(mri_data)
from mri_data import file_manager as fm
from mri_data import utils

pd.options.mode.copy_on_write = True

In [36]:
drive_root = fm.get_drive_root()
dataroot = drive_root / "3Tpioneer_bids"
labelroot = drive_root / "srs-9/3Tpioneer_bids_predictions"
data_dir = Path("/home/srs-9/Projects/ms_mri/analysis/paper1/data0")
df = pd.read_csv(data_dir / "manual_labels.csv")
df.set_index("subid", inplace=True)

df_full = pd.read_csv(data_dir / "t1_2_data_full.csv", index_col="subid")

In [12]:
for i, row in df.iterrows():
    scanroot = dataroot / str(row['sub-ses']) 
    images = []
    for im in ["t1.nii.gz", "t1_gd.nii.gz", "flair.nii.gz"]:
        if (scanroot / im).is_file():
            images.append(str(scanroot / im))
    labels = []
    for lab in ["pineal-SRS.nii.gz", "pineal-SRS_T1.nii.gz"]:
        if (scanroot / lab).is_file():
            labels.append(str(scanroot / lab))
    
    cmd = utils.open_itksnap_workspace_cmd(images, labels)
    df.loc[i, "cmd"] = cmd

In [ ]:
sub = 1453
cmd = df.loc[sub, "cmd"]
print(cmd)
pyperclip.copy(cmd)

In [34]:
for i, row in df_full.iterrows():
    scanroot = dataroot / str(row['sub-ses']) 
    images = []
    for im in ["t1.nii.gz", "t1_gd.nii.gz", "flair.nii.gz"]:
        if (scanroot / im).is_file():
            images.append(str(scanroot / im))
    if len(images) == 0:
        continue
    labels = []
    for lab in ["pineal-SRS.nii.gz", "pineal-SRS_T1.nii.gz"]:
        if (scanroot / lab).is_file():
            labels.append(str(scanroot / lab))
    
    cmd = utils.open_itksnap_workspace_cmd(images, labels, win=True)
    df_full.loc[i, "cmd"] = cmd

In [32]:
df_check = df_full[["pineal_volume"]]
df_check.sort_values(by="pineal_volume", inplace=True)